In [1]:
import sys, os
import gc
import glob
from time import sleep
from time import time
if '../' not in sys.path:
    sys.path.insert(0, '../')

from modules.stacks import Stack
    
import matplotlib.pyplot as plt
import numpy as np

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
""" Set parameters """

preprocessing_kws = dict(median_radius=2,
                         gaussian_sigma=(2, 2),
                         clip_limit=0.03, clip_factor=20)
seed_kws=dict(sigma=2, min_distance=1)
seg_kws=dict()
segmentation_kw = dict(preprocessing_kws=preprocessing_kws, 
          seed_kws=seed_kws, seg_kws=seg_kws,
          min_segment_area=250)

fg_kw = dict(bandwidth=100, n=5)

annotation_kw = dict(q=95,
                     channel='r_normalized',
                     weighted=True,
                     fg_only=True,
                     upper_bound=90)

save_kw = dict(segmentation=True, image=True, foreground=True, annotation=True, dpi=100)
kw = dict(segmentation_kw=segmentation_kw, fg_kw=fg_kw, annotation_kw=annotation_kw, save_kw=save_kw)

In [3]:
control_slices = '/Volumes/bernasek_t5/pnt_manuscript/microscopy/yan_clones/control/slices/'
mutant_slices = '/Volumes/bernasek_t5/pnt_manuscript/microscopy/yan_clones/perturbation/slices/'
genotypes = os.listdir(control_slices)

In [5]:
import gc
import logging
logging.basicConfig(level=logging.INFO, #filename='segmentation.log', 
                    format='%(asctime)s - %(levelname)s - %(message)s')

def run(stack_path, save=True, **kw):
    disc_name = stack_path.split('_')[-1].strip('.tif')
    logging.info('Starting stack {:s}'.format(disc_name))
    stack = Stack(stack_path)
    try:
        stack.segment(bg='b', save=save, **kw)
    except Exception as e:
        logging.error(e)
        e.__traceback__ = None
    logging.info('Completed stack {:s}'.format(disc_name))

In [6]:
for genotype in os.listdir(control_slices)[::-1]:
    logging.info('Starting genotype {:s}'.format(genotype))
    experiment_path = os.path.join(control_slices, genotype)
    stack_paths = glob.glob(os.path.join(experiment_path, '*[0-9].tif'))        
    for stack_path in stack_paths:
        run(stack_path, save=True, **kw)
    logging.info('Finished genotype {:s}'.format(genotype))
        
for genotype in os.listdir(mutant_slices):
    logging.info('Starting genotype {:s}'.format(genotype))
    experiment_path = os.path.join(mutant_slices, genotype)
    stack_paths = glob.glob(os.path.join(experiment_path, '*[0-9].tif'))
    for stack_path in stack_paths:
        run(stack_path, save=True, **kw)
    logging.info('Finished genotype {:s}'.format(genotype))

logging.info('Finished all stacks.')

2018-07-21 00:28:19,935 - INFO - Starting genotype w-eyFLP_PntGFP_FRT40A_PntGFP_Ubi-mRFPnls_FRT40A
2018-07-21 00:28:19,937 - INFO - Starting stack FRT40A/29
2018-07-21 00:28:51,380 - INFO - Completed stack FRT40A/29
2018-07-21 00:28:51,397 - INFO - Starting stack FRT40A/30
2018-07-21 00:31:04,999 - INFO - Completed stack FRT40A/30
2018-07-21 00:31:05,073 - INFO - Starting stack FRT40A/31
2018-07-21 00:31:33,794 - INFO - Completed stack FRT40A/31
2018-07-21 00:31:33,811 - INFO - Starting stack FRT40A/55
2018-07-21 00:33:01,097 - INFO - Completed stack FRT40A/55
2018-07-21 00:33:01,143 - INFO - Starting stack FRT40A/56
2018-07-21 00:33:30,206 - INFO - Completed stack FRT40A/56
2018-07-21 00:33:30,223 - INFO - Starting stack FRT40A/28
2018-07-21 00:34:12,898 - INFO - Completed stack FRT40A/28
2018-07-21 00:34:12,922 - INFO - Finished genotype w-eyFLP_PntGFP_FRT40A_PntGFP_Ubi-mRFPnls_FRT40A
2018-07-21 00:34:12,923 - INFO - Starting genotype Control_w-eyFLP_PntGFP_FRT40A_CyO
2018-07-21 00:3

In [15]:
output = []

def parse(experiment, genotype, stack_path):
    stack = Stack(stack_path)
    for layer in range(stack.depth):
        record = dict(experiment=experiment, genotype=genotype, disc=stack.disc_name, layer=layer)
        output.append(record)
    

In [16]:
for genotype in os.listdir(control_slices)[::-1]:
    experiment_path = os.path.join(control_slices, genotype)
    stack_paths = glob.glob(os.path.join(experiment_path, '*[0-9].tif'))        
    for stack_path in stack_paths:
        parse('control', genotype, stack_path)
        
for genotype in os.listdir(mutant_slices):
    experiment_path = os.path.join(mutant_slices, genotype)
    stack_paths = glob.glob(os.path.join(experiment_path, '*[0-9].tif'))
    for stack_path in stack_paths:
        parse('perturbation', genotype, stack_path)

In [18]:
import pandas as pd

In [20]:
df = pd.DataFrame.from_records(output)

In [24]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('layers.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='layers')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [12]:
stack.genotype_path

'/Volumes/bernasek_t5/pnt_manuscript/microscopy/yan_clones/perturbation/slices/Control_w-eyFLP_Ubi-mRFPnls_FRT40A_Cyo'

In [ ]:
def run_diagnostic(stack_path, save=True, **kw):
    disc_name = stack_path.split('_')[-1].strip('.tif')
    stack = Stack(stack_path)
    stack.segment(bg='b', save=save, **kw)

In [6]:
for genotype in os.listdir(control_slices)[::-1]:
    logging.info('Starting genotype {:s}'.format(genotype))
    experiment_path = os.path.join(control_slices, genotype)
    stack_paths = glob.glob(os.path.join(experiment_path, '*[0-9].tif'))    
    
    break

In [10]:
run_diagnostic(stack_paths[1], save=True, **kw)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [12]:
# # controls
# control_genotype = 'w-eyFLP_PntGFP_FRT40A_PntGFP_Ubi-mRFPnls_FRT40A'
# overall_start = time()
# logging.info('Starting genotype {:s}'.format(control_genotype))
# experiment_path = os.path.join(control_slices, control_genotype)
# stack_paths = glob.glob(os.path.join(experiment_path, 'control*[0-9].tif'))        
# for stack_path in stack_paths:
#     run(stack_path, save=True, **kw)
# logging.info('Finished genotype {:s}'.format(control_genotype))

# mutants
mutant_genotype = 'w-eyFLP_PntGFP_YanE833_FRT40A_PntGFP_Ubi-mRFPnls_FRT40A'
logging.info('Starting genotype {:s}'.format(mutant_genotype))
experiment_path = os.path.join(mutant_slices, mutant_genotype)
stack_paths = glob.glob(os.path.join(experiment_path, 'perturbation*[0-9].tif'))
for stack_path in stack_paths[13:]:
    run(stack_path, save=True, **kw)
    gc.collect()
    sleep(30)
logging.info('Finished genotype {:s}'.format(mutant_genotype))
logging.info('Finished all stacks.')

In [13]:
from pympler import summary, muppy
all_objects = muppy.get_objects()
summarized = summary.summarize(all_objects)
summarized = sorted(summarized, key=lambda x: x[-1], reverse=True)
summary.print_(summarized[:20], limit=5) 

                       types |   # objects |   total size
============================ | =========== | ============
                 <class 'str |       96225 |     17.91 MB
                <class 'dict |       16509 |     10.62 MB
  <class '_io.BufferedWriter |           4 |      4.38 MB
                <class 'code |       30480 |      4.19 MB
                <class 'type |        3397 |      3.50 MB
